In [2]:
#libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

Webscrape info for each city. Make sure to also scrape for other populated cities in spain.

Current Cities:
* Barcelona
* Madrid
* Seville
* Valencia

New Cities
* Vigo
* Malaga

Variable List
* Population: https://worldpopulationreview.com/cities/spain
* Living Cost: https://www.numbeo.com/cost-of-living/country_result.jsp?country=Spain


**Webscraping Population for each city**

In [4]:
#requests
pop_response = requests.get("https://worldpopulationreview.com/cities/spain")

#soup
pop_soup = BeautifulSoup(pop_response.content, "html.parser")

In [12]:
pop_table = pop_soup.find("table",
                    attrs={
                      "class": "wpr-table !border-none"}
                  )

#check to see its only 1 table
#len(pop_table)

In [21]:
# initialize an empty list
rows = []

# iterate over all rows in the city table
#we do 1: b/c there is only 1 header row in this data
for city in pop_table.find_all("tr")[1:]:

    # Get all the cells (<td>) in the row.
    cities = city.find_all("th")
    pops = city.find_all("td")
    
    # can just look at table and see what obs we want along and pick that indexes b/c its in a weirdish order
    city_tag = cities[0]
    city_value = city_tag.text.strip()

    #find # stores in 2024
    population_tag = pops[0]
    population_value = population_tag.text.strip()


    # Append this data.
    rows.append({
            "city": city_value,
            "population": population_value
            #"census_2010": census_2010
     })


pop_df = pd.DataFrame(rows)

**Webscraping Living Cost for each city**

In [34]:
#requests
liv_response = requests.get("https://www.numbeo.com/cost-of-living/country_result.jsp?country=Spain")

#soup
liv_soup = BeautifulSoup(liv_response.content, "html.parser")

In [39]:
liv_table = liv_soup.find("table",
                    attrs={
                      "id": "t2"}
                  )

#check to see its only 1 table
len(liv_table)


5

In [49]:
# initialize an empty list
rows = []

# iterate over all rows in the city table
#we do 1: b/c there is only 1 header row in this data
for city in liv_table.find_all("tr")[1:]:

    # Get all the cells (<td>) in the row.
    cells = city.find_all("td")
    
    # can just look at table and see what obs we want along and pick that indexes b/c its in a weirdish order
    city_tag = cells[1]
    city_value = city_tag.text.strip()

    #cost of living index
    living_index_tag = cells[2]
    living_index_value = living_index_tag.text.strip()

    #rent index
    rent_index_tag = cells[3]
    rent_index_value = rent_index_tag.text.strip()

    #cost of living + rent index
    living_plus_rent_index_tag = cells[4]
    living_plus_rent_index_value = living_plus_rent_index_tag.text.strip()

    #groceries index
    groceries_index_tag = cells[5]
    groceries_index_value = groceries_index_tag.text.strip()

    #restuarant price index
    restuarant_index_tag = cells[6]
    restuarant_index_value = restuarant_index_tag.text.strip()

    #local purchasing power index
    local_purchase_power_index_tag = cells[7]
    local_purchase_power_index_value = local_purchase_power_index_tag.text.strip()

    # Append this data.
    rows.append({
            "city": city_value,
            "cost_of_living_index": living_index_value,
            "rent_index": rent_index_value,
            "living_plus_rent_index": living_plus_rent_index_value,
            "groceries_index": groceries_index_value,
            "restuarant_index": restuarant_index_value,
            "local_purchasing_power_index": local_purchase_power_index_value
     })


liv_df = pd.DataFrame(rows)

In [50]:
#final tweaks to make sure the dataset matches based on city name
liv_df['city'][3] = 'Sevilla'

<positron-console-cell-50>:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [51]:
#merge datasets
final_df = pd.merge(pop_df, liv_df, how='inner', on=['city'])


In [59]:
#convert all numeric variables to numeric:

#population variable
for i in range(len(final_df)):
   final_df['population'][i] = final_df['population'][i].replace(",", "")

final_df['population'] = pd.to_numeric(final_df['population'])

#all other variables
final_df['cost_of_living_index'] = pd.to_numeric(final_df['cost_of_living_index'])
final_df['rent_index'] = pd.to_numeric(final_df['rent_index'])
final_df['living_plus_rent_index'] = pd.to_numeric(final_df['living_plus_rent_index'])
final_df['groceries_index'] = pd.to_numeric(final_df['groceries_index'])
final_df['restuarant_index'] = pd.to_numeric(final_df['restuarant_index'])
final_df['local_purchasing_power_index'] = pd.to_numeric(final_df['local_purchasing_power_index'])




<positron-console-cell-59>:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<positron-console-cell-59>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/